Name: Karthik Venugopal

USC ID: 9330529336

github username: Karthikvenugopal

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import shutil
import random
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
def separating_dataset_train_test(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-5, "Ratios must sum to 1"
    random.seed(seed)

    for class_name in filter(lambda x: os.path.isdir(os.path.join(source_dir, x)), os.listdir(source_dir)):
        class_path = os.path.join(source_dir, class_name)
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)

        n_total = len(images)
        n_train = int(train_ratio * n_total)
        n_val = int(val_ratio * n_total)

        splits = {
            'train': images[:n_train],
            'val': images[n_train:n_train + n_val],
            'test': images[n_train + n_val:]
        }

        for split, split_images in splits.items():
            split_dir = os.path.join(dest_dir, split, class_name)
            os.makedirs(split_dir, exist_ok=True)
            for img in split_images:
                shutil.copy2(os.path.join(class_path, img), os.path.join(split_dir, img))

    print(f"Dataset split completed and saved to {dest_dir}")


In [3]:
separating_dataset_train_test(source_dir="RealWaste", dest_dir="SeparatedWaste", train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42)

Dataset split completed and saved to SeparatedWaste


In [4]:
base_dir = "SeparatedWaste"
sub_dirs = {split: os.path.join(base_dir, split) for split in ["train", "val", "test"]}

img_size = (224, 224)
batch_size = 5

preprocess_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)

data_gens = {
    split: preprocess_gen.flow_from_directory(
        sub_dirs[split],
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=(split == "train")
    )
    for split in ["train", "val", "test"]
}

train_gen = data_gens["train"]
val_gen = data_gens["val"]
test_gen = data_gens["test"]
epochs = 100


Found 3323 images belonging to 9 classes.
Found 710 images belonging to 9 classes.
Found 719 images belonging to 9 classes.


In [5]:
img_height, img_width = img_size
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=40,
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model_resnet50.h5',
    monitor='val_loss',
    save_best_only=True
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


665/665 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.3798 - loss: 7.8173

665/665 ━━━━━━━━━━━━━━━━━━━━ 121s 180ms/step - accuracy: 0.3800 - loss: 7.8159 - val_accuracy: 0.6014 - val_loss: 5.3412
Epoch 2/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.5766 - loss: 5.0556

665/665 ━━━━━━━━━━━━━━━━━━━━ 131s 197ms/step - accuracy: 0.5766 - loss: 5.0550 - val_accuracy: 0.6944 - val_loss: 3.6341
Epoch 3/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.6176 - loss: 3.6278

...

665/665 ━━━━━━━━━━━━━━━━━━━━ 169s 254ms/step - accuracy: 0.7183 - loss: 1.1747 - val_accuracy: 0.7592 - val_loss: 0.9952
Epoch 60/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 164s 246ms/step - accuracy: 0.7320 - loss: 1.1328 - val_accuracy: 0.7563 - val_loss: 1.0628
Epoch 61/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 165s 249ms/step - accuracy: 0.7248 - loss: 1.1283 - val_accuracy: 0.7690 - val_loss: 1.0248
Epoch 62/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 167s 251ms/step - accuracy: 0.7298 - loss: 1.1375 - val_accuracy: 0.7690 - val_loss: 1.0038
Epoch 63/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 169s 254ms/step - accuracy: 0.7415 - loss: 1.0824 - val_accuracy: 0.7099 - val_loss: 1.1605
Epoch 64/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.7354 - loss: 1.1077

665/665 ━━━━━━━━━━━━━━━━━━━━ 171s 257ms/step - accuracy: 0.7354 - loss: 1.1077 - val_accuracy: 0.7704 - val_loss: 0.9827
Epoch 65/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.7428 - loss: 1.0786

665/665 ━━━━━━━━━━━━━━━━━━━━ 175s 263ms/step - accuracy: 0.7428 - loss: 1.0786 - val_accuracy: 0.7775 - val_loss: 0.9766
Epoch 66/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.7518 - loss: 1.0560

665/665 ━━━━━━━━━━━━━━━━━━━━ 172s 259ms/step - accuracy: 0.7517 - loss: 1.0561 - val_accuracy: 0.7704 - val_loss: 0.9655
Epoch 67/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 170s 256ms/step - accuracy: 0.7504 - loss: 1.0824 - val_accuracy: 0.6817 - val_loss: 1.4297
Epoch 68/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 172s 258ms/step - accuracy: 0.7225 - loss: 1.1686 - val_accuracy: 0.7141 - val_loss: 1.1641
Epoch 69/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 172s 259ms/step - accuracy: 0.7568 - loss: 1.0761 - val_accuracy: 0.7789 - val_loss: 0.9950
Epoch 70/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 171s 258ms/step - accuracy: 0.7507 - loss: 1.0591 - val_accuracy: 0.7239 - val_loss: 1.1530
Epoch 71/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 171s 257ms/step - accuracy: 0.7555 - loss: 1.0551 - val_accuracy: 0.7056 - val_loss: 1.1826
Epoch 72/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 170s 255ms/step - accuracy: 0.7211 - loss: 1.1413 - val_accuracy: 0.6718 - val_loss: 1.2952
Epoch 73/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 172s 258ms/step - accuracy: 0.717

665/665 ━━━━━━━━━━━━━━━━━━━━ 167s 252ms/step - accuracy: 0.7279 - loss: 1.1278 - val_accuracy: 0.7901 - val_loss: 0.9378
Epoch 100/100
665/665 ━━━━━━━━━━━━━━━━━━━━ 168s 253ms/step - accuracy: 0.7232 - loss: 1.1353 - val_accuracy: 0.6577 - val_loss: 1.2886


In [7]:
model.save('final_resnet50_model.keras')